In [19]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup


from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity="all"

Location:
- Baga beach
- Panjim
- Calangute
- Anjuna
- Vagator
- South Goa


h_fn- function to extact hotesl from data

In [20]:
# function to convert into soup object

def soup_ob(page):
    data=BeautifulSoup(page.text,'html.parser')
    return data

In [21]:
# function to extract hotel names

def hotel_fn(data):
    hotel=data.find_all('h3',class_="listingHotelDescription__hotelName d-textEllipsis")
    hotel_list=[]
    for i in hotel:
        tag=i.text
        hotel_list.append(tag)
    return hotel_list

In [22]:
# function to extract rating

def rating_fn(data):
    rating = data.find_all('div',class_='hotelRating')
    rating_list = []
    for i in rating:
        a = i.find_all('span')
        b = a[0].text
        if re.match(r'\d\.\d',b):
            rating_list.append(float(b))
        else:
            rating_list.append(None)
    return rating_list

In [23]:
# function to extract price

def price_fn(data):
    price=data.find_all('span',class_="listingPrice__finalPrice")
    price_list=[]
    for i in price:
        tag=i.text
        tag1=tag.replace('₹','')
        price_list.append(int(tag1))
    return price_list


In [24]:
# function to count no.of reviews

def count_fn(data):
    count=data.find_all('div', class_="hotelRating")
    count_list=[]
    for i in count:
        tag=i.find_all('meta', attrs={'itemprop':'reviewCount'})
        if tag==[]:
            count_list.append(None)
        else:
            tag1=(tag[0].get('content'))
            count_list.append(int(tag1))
    return count_list

In [25]:
# function for original price

def actualprice_fn(data):
    original=data.find_all('span',class_="listingPrice__slashedPrice d-body-lg")
    original_list=[]
    for i in original:
        tag=i.text
        tag1=tag.replace('₹','')
        original_list.append(int(tag1))
    return original_list

In [26]:
# function for discount

def discount_fn(data):
    discount=data.find_all('span',class_="listingPrice__percentage")
    discount_list=[]
    for i in discount:
        tag=i.text
        if re.match(r'\d+.\s\w+',tag):
            tag1=tag.replace('% off','')
            discount_list.append(int(tag1))
    return discount_list

In [27]:
def amenities_fn(data):
    amenities_data=data.find_all('div',class_="amenityWrapper")
    amenities_list=[]
    for i in amenities_data:
        tag=i.find_all('span',class_="d-body-sm d-textEllipsis")
        li = []
        for j in tag:
            li.append(j.text)
        amenities_list.append(','.join(li))
    return amenities_list

In [28]:
def AmenityWrapper_fn(data):
    Amenity_data=data.find_all('div',class_="amenityWrapper")
    list1 = []
    for i in Amenity_data:
        b=i.find_all('span',class_="d-body-sm d-textEllipsis")
        li = []
        for j in b:
            li.append(j.text)
        list1.append(li)

    parking=[]; card=[]; AC=[]; breakfast=[]; restaurant=[]; kitchen=[]; TV=[]; CCTV=[]; firstaid=[]; power=[]; bar=[]; bathroom=[]

    for i in list1:
        if ' Parking Facility ' in i:
            parking.append('Yes')
        else:
            parking.append('No')
        if ' AC ' in i:
            AC.append('Yes')
        else:
            AC.append('No')
        if ' Card Payment ' in i:
            card.append('Yes')
        else:
            card.append('No')
        if ' Complimentary Breakfast ' in i:
            breakfast.append('Yes')
        else:
            breakfast.append('No')
        if 'In-house Restaurant ' in i:
            restaurant.append('Yes')
        else:
            restaurant.append('No')
        if  ' Kitchen ' in i:
            kitchen.append('Yes')
        else:
            kitchen.append('No')
        if ' TV ' in i:
            TV.append('Yes')
        else:
            TV.append('No')
        if ' CCTV Cameras ' in i:
            CCTV.append('Yes')
        else:
            CCTV.append('No')
        if ' First Aid ' in i:
            firstaid.append('Yes')
        else:
            firstaid.append('No')
        if '  Bar ' in i:
            bar.append('Yes')
        else:
            bar.append('No')
        if ' Attached Bathroom ' in i:
            bathroom.append('Yes')
        else:
            bathroom.append('No')
        if ' Power backup ' in i:
            power.append('Yes')
        else:
            power.append('No')
            
    return parking,card,AC,breakfast,restaurant,kitchen,TV,CCTV,firstaid,power,bar,bathroom

In [29]:
#for i in list1:
#   if i[0] == " Parking Facility ":
#      Parking_bagabeach.append('Yes')
#   elif i[1] == " Parking Facility ":
#      Parking_bagabeach.append('Yes')
#   elif i[2] == " Parking Facility ":
#      Parking_bagabeach.append('Yes')
#   else:
#      Parking_bagabeach.append('No')

In [30]:
def extract(data):
    
    h=hotel_fn(data)
    r=rating_fn(data)
    c=count_fn(data)
    ap=actualprice_fn(data)
    d=discount_fn(data)
    p=price_fn(data)
    am=amenities_fn(data)
    
    parking,card,AC,breakfast,restaurant,kitchen,TV,CCTV,firstaid,power,bar,bathroom=AmenityWrapper_fn(data)
    
    return h,r,c,ap,d,p,am,parking,card,AC,breakfast,restaurant,kitchen,TV,CCTV,firstaid,power,bar,bathroom


In [31]:
# Creating DataFrame

def df_fn(data):  
    
    location=input()    
    
    h,r,c,ap,d,p,am,parking,card,AC,breakfast,restaurant,kitchen,TV,CCTV,firstaid,power,bar,bathroom=extract(data)
    
    df=pd.DataFrame()    
    df['Hotel']=h
    df['Rating']=r
    df['Count']=c
    df['Location']=location
    df['Actual Price(₹)']=ap
    df['Discount(% off)']=d
    df['Price(₹)']=p  
    df['Amenities']=am
    df["Parking"]=parking
    df['Card Payment']=card
    df['AC']=AC
    df['BreakFast']=breakfast
    df['In-house Restaurant']=restaurant
    df['Kitchen']=kitchen
    df['TV']=TV
    df['CCTV Cameras']=CCTV
    df['First Aid']=firstaid
    df['Power backup']=power
    df['Bar']=bar
    df['Attached Bathroom']=bathroom
    
    return df

In [32]:
# space for more functions.

# URLs 

In [33]:
url_bagabeach="""https://www.oyorooms.com/hotels-in-baga-beach-north-goa-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.5556936&filters%5Bcoordinates%5D%5Blongitude%5D=73.751647&roomConfig=2"""
page_bagabeach=requests.get(url_bagabeach)
data_bagabeach=soup_ob(page_bagabeach)

In [34]:
url_panjim="""https://www.oyorooms.com/hotels-in-panjim-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.4831457&filters%5Bcoordinates%5D%5Blongitude%5D=73.8212075&roomConfig=2"""
page_panjim=requests.get(url_panjim)
data_panjim=soup_ob(page_panjim)

In [35]:
url_calangute="""https://www.oyorooms.com/hotels-in-calangute-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.549441&filters%5Bcoordinates%5D%5Blongitude%5D=73.7534857&gclid=EAIaIQobChMIodqX3Lvk5wIVgg4rCh01Cw0vEAAYASAAEgJxpfD_BwE&roomConfig=2&utm_campaign=India_SEM_Brand_generic&utm_medium=cpc&utm_source=google"""
page_calangute=requests.get(url_calangute)
data_calangute=soup_ob(page_calangute)

In [36]:
url_mapusa="""https://www.oyorooms.com/hotels-in-mapusa-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.6002273&filters%5Bcoordinates%5D%5Blongitude%5D=73.81249839999998&gclid=EAIaIQobChMIodqX3Lvk5wIVgg4rCh01Cw0vEAAYASAAEgJxpfD_BwE&roomConfig=2&utm_campaign=India_SEM_Brand_generic&utm_medium=cpc&utm_source=google"""
page_mapusa=requests.get(url_mapusa)
data_mapusa=soup_ob(page_mapusa)

In [37]:
url_dabolimairport="""https://www.oyorooms.com/hotels-in-dabolim-airport-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.3861332&filters%5Bcoordinates%5D%5Blongitude%5D=73.83900449999999&roomConfig=2"""
page_dabolimairport=requests.get(url_dabolimairport)
data_dabolimairport=soup_ob(page_dabolimairport)

In [38]:
url_vascodagama="""https://www.oyorooms.com/hotels-in-vasco-da-gama-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.3860329&filters%5Bcoordinates%5D%5Blongitude%5D=73.8440398&roomConfig=2"""
page_vascodagama=requests.get(url_vascodagama)
data_vascodagama=soup_ob(page_vascodagama)

In [39]:
url_madgaon_rs="""https://www.oyorooms.com/hotels-in-madgaon-railway-station-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.2832187&filters%5Bcoordinates%5D%5Blongitude%5D=73.986191&roomConfig=2"""
page_madgaon_rs=requests.get(url_madgaon_rs)
data_madgaon_rs=soup_ob(page_madgaon_rs)

In [40]:
url_morjim="""https://www.oyorooms.com/hotels-in-morjim-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.63162&filters%5Bcoordinates%5D%5Blongitude%5D=73.73896&roomConfig=2"""
page_morjim=requests.get(url_morjim)
data_morjim=soup_ob(page_morjim)

In [41]:
url_panaji="""https://www.oyorooms.com/hotels-in-panaji-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.4831457&filters%5Bcoordinates%5D%5Blongitude%5D=73.8212075&roomConfig=2"""
page_panaji=requests.get(url_panaji)
data_panaji=soup_ob(page_panaji)

In [42]:
url_vascodagama_rs="""https://www.oyorooms.com/hotels-in-vasco-da-gama-railway-station-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.3860329&filters%5Bcoordinates%5D%5Blongitude%5D=73.8440398&roomConfig=2"""
page_vascodagama_rs=requests.get(url_vascodagama_rs)
data_vascodagama_rs=soup_ob(page_vascodagama_rs)

In [43]:
url_ponda="""https://www.oyorooms.com/hotels-in-ponda-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.3991304&filters%5Bcoordinates%5D%5Blongitude%5D=74.01238510000007&roomConfig=2"""
page_ponda=requests.get(url_ponda)
data_ponda=soup_ob(page_ponda)

In [44]:
url_candolim="""https://www.oyorooms.com/hotels-in-candolim-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.5183861&filters%5Bcoordinates%5D%5Blongitude%5D=73.7654787&roomConfig=2"""
page_candolim=requests.get(url_candolim)
data_candolim=soup_ob(page_candolim)

In [45]:
url_donapaula="""https://www.oyorooms.com/hotels-in-dona-paula-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.4608099&filters%5Bcoordinates%5D%5Blongitude%5D=73.8095574&roomConfig=2"""
page_donapaula=requests.get(url_donapaula)
data_donapaula=soup_ob(page_donapaula)

In [46]:
url_varca="""https://www.oyorooms.com/hotels-in-varca-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.2323661&filters%5Bcoordinates%5D%5Blongitude%5D=73.9431128&roomConfig=2"""
page_varca=requests.get(url_varca)
data_varca=soup_ob(page_varca)

In [47]:
# Not using

#url_arambol="""https://www.oyorooms.com/hotels-in-arambol-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.68713&filters%5Bcoordinates%5D%5Blongitude%5D=73.72131&roomConfig=2"""
#page_arambol=requests.get(url_arambol)
#data_arambol=soup_ob(page_arambol)

In [48]:
url_colva="""https://www.oyorooms.com/hotels-in-colva-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.275651&filters%5Bcoordinates%5D%5Blongitude%5D=73.913727&roomConfig=2"""
page_colva=requests.get(url_colva)
data_colva=soup_ob(page_colva)

In [49]:
url_vagator="""https://www.oyorooms.com/hotels-in-vagator-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.59886&filters%5Bcoordinates%5D%5Blongitude%5D=73.75661&roomConfig=2"""
page_vagator=requests.get(url_vagator)
data_vagator=soup_ob(page_vagator)

In [50]:
url_anjuna="""https://www.oyorooms.com/hotels-in-anjuna-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.5809248&filters%5Bcoordinates%5D%5Blongitude%5D=73.74484370000005&roomConfig=2"""
page_anjuna=requests.get(url_anjuna)
data_anjuna=soup_ob(page_anjuna)

In [51]:
url_southgoa="""https://www.oyorooms.com/hotels-in-south-goa-goa/?checkin=08%2F03%2F2020&checkout=09%2F03%2F2020&filters%5Bcoordinates%5D%5Blatitude%5D=15.2993265&filters%5Bcoordinates%5D%5Blongitude%5D=74.123996&roomConfig=2"""
page_southgoa=requests.get(url_southgoa)
data_southgoa=soup_ob(page_southgoa)

### Hotels from Baga Beach

In [52]:
df1=df_fn(data_bagabeach)

Baga Beach


In [53]:
df1

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,Collection O 30082 Baga Alchemy Creek,4.0,517,Baga Beach,5651,40,3391,"Parking Facility , AC , Card Payment",Yes,Yes,Yes,No,No,No,No,No,No,No,No,No
1,OYO 7288 Traveller Guest House,4.4,136,Baga Beach,3568,40,2141,"AC , TV , CCTV Cameras",No,No,Yes,No,No,No,Yes,Yes,No,No,No,No
2,Capital O 64180 Adam's Beach Resort,4.8,252,Baga Beach,5026,40,3016,"AC , First Aid , TV",No,No,Yes,No,No,No,Yes,No,Yes,No,No,No
3,OYO Home 65590 Titos Lane 1 Baga Beach,4.0,199,Baga Beach,4165,40,2499,"Attached Bathroom , CCTV Cameras , Power backup",No,No,No,No,No,No,No,Yes,No,Yes,No,Yes
4,OYO 3385 Hotel Baga Bay,4.8,291,Baga Beach,3623,40,2174,"In-house Restaurant , Kitchen , AC",No,No,Yes,No,No,Yes,No,No,No,No,No,No
5,OYO Home 66565 Spacious Duplex Calangute,4.3,46,Baga Beach,3055,40,1833,"Parking Facility , Attached Bathroom , CCTV C...",Yes,No,No,No,No,No,No,Yes,No,No,No,Yes
6,OYO Home 66780 Spacious Studios Baga,4.6,157,Baga Beach,4535,40,2721,"Attached Bathroom , CCTV Cameras , Power backup",No,No,No,No,No,No,No,Yes,No,Yes,No,Yes
7,OYO 5242 Zinho's,4.6,354,Baga Beach,6518,40,3911,"Kitchen , CCTV Cameras , Geyser",No,No,No,No,No,Yes,No,Yes,No,No,No,No
8,OYO Townhouse 065 Calangute Aguada,4.3,759,Baga Beach,5417,40,3250,"CCTV Cameras , Complimentary Breakfast , In-h...",No,No,No,Yes,No,No,No,Yes,No,No,No,No
9,OYO Flagship 17197 Hotel Horizon Calangute,4.4,1031,Baga Beach,4772,40,2863,"AC , TV , Bar",No,No,Yes,No,No,No,Yes,No,No,No,No,No


In [54]:
r = rating_fn(data_bagabeach)

In [55]:
type(r[0])

float

In [56]:
p = price_fn(data_bagabeach)

In [57]:
type(p[0])

int

In [58]:
a= actualprice_fn(data_bagabeach)
type(a[0])

int

In [59]:
d =discount_fn(data_bagabeach)
type(d[0])

int

In [60]:
c=count_fn(data_bagabeach)
type(c[0])

int

In [61]:
# space for more columns

## Hotels from Panjim

In [62]:
df2=df_fn(data_panjim)

Panjim


In [63]:
df2

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,Collection O 22006 Paradise Inn,4.7,1114,Panjim,3463,45,1891,"CCTV Cameras , Reception , Free Wifi",No,No,No,No,No,No,No,Yes,No,No,No,No
1,Collection O 50349 Hotel Sunheads Nalanda Colony,4.5,579,Panjim,3081,40,1849,"AC , TV , Attached Bathroom",No,No,Yes,No,No,No,Yes,No,No,No,No,Yes
2,Collection O 22247 Casa Paradiso,4.7,1142,Panjim,3070,40,1842,"Power backup , CCTV Cameras",No,No,No,No,No,No,No,Yes,No,Yes,No,No
3,Collection O 18786 Hotel Sona Panaji,4.5,1499,Panjim,3214,44,1803,"AC , TV , CCTV Cameras",No,No,Yes,No,No,No,Yes,Yes,No,No,No,No
4,OYO 11845 Hotel Victoria,4.5,381,Panjim,3078,40,1847,"Elevator , AC , TV",No,No,Yes,No,No,No,Yes,No,No,No,No,No
5,OYO 9518 Hotel Aqua Mira,4.0,145,Panjim,2655,40,1593,"Parking Facility , CCTV Cameras , Geyser",Yes,No,No,No,No,No,No,Yes,No,No,No,No
6,OYO 2863 Hotel 4 Pillar's,4.4,185,Panjim,2857,40,1714,"Complimentary Breakfast , AC , Power backup",No,No,Yes,Yes,No,No,No,No,No,Yes,No,No
7,OYO 35596 Hotel Amar,4.5,89,Panjim,2177,40,1306,"Parking Facility , First Aid , Attached Bathr...",Yes,No,No,No,No,No,No,No,Yes,No,No,Yes
8,Capital O 65108 Viva Goa International,4.1,93,Panjim,5498,40,3299,"Parking Facility , First Aid , Attached Bathr...",Yes,No,No,No,No,No,No,No,Yes,No,No,Yes
9,OYO 14986 Charming 1 BHK,3.7,146,Panjim,5608,40,3365,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No


In [64]:
# space for more columns

## Hotels from Calangute

In [65]:
df3=df_fn(data_calangute)

Calangute


In [66]:
df3

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,Collection O 30082 Baga Alchemy Creek,4.0,517.0,Calangute,5651,40,3391,"Parking Facility , AC , Card Payment",Yes,Yes,Yes,No,No,No,No,No,No,No,No,No
1,OYO 7288 Traveller Guest House,4.4,136.0,Calangute,3568,40,2141,"AC , TV , CCTV Cameras",No,No,Yes,No,No,No,Yes,Yes,No,No,No,No
2,OYO Home 66565 Spacious Duplex Calangute,4.3,46.0,Calangute,3055,40,1833,"Parking Facility , Attached Bathroom , CCTV C...",Yes,No,No,No,No,No,No,Yes,No,No,No,Yes
3,Capital O 64180 Adam's Beach Resort,4.8,252.0,Calangute,5026,40,3016,"AC , First Aid , TV",No,No,Yes,No,No,No,Yes,No,Yes,No,No,No
4,OYO Townhouse 065 Calangute Aguada,4.3,759.0,Calangute,5417,40,3250,"CCTV Cameras , Complimentary Breakfast , In-h...",No,No,No,Yes,No,No,No,Yes,No,No,No,No
5,OYO Home 65590 Titos Lane 1 Baga Beach,4.0,199.0,Calangute,4165,40,2499,"Attached Bathroom , CCTV Cameras , Power backup",No,No,No,No,No,No,No,Yes,No,Yes,No,Yes
6,OYO 3385 Hotel Baga Bay,4.8,291.0,Calangute,3623,40,2174,"In-house Restaurant , Kitchen , AC",No,No,Yes,No,No,Yes,No,No,No,No,No,No
7,OYO Flagship 17197 Hotel Horizon Calangute,4.4,1031.0,Calangute,4772,40,2863,"AC , TV , Bar",No,No,Yes,No,No,No,Yes,No,No,No,No,No
8,OYO Townhouse 041 Calangute,4.4,959.0,Calangute,5106,40,3064,"Parking Facility , Complimentary Breakfast , ...",Yes,No,No,Yes,No,No,No,No,No,No,No,No
9,OYO Home 66780 Spacious Studios Baga,4.6,157.0,Calangute,4535,40,2721,"Attached Bathroom , CCTV Cameras , Power backup",No,No,No,No,No,No,No,Yes,No,Yes,No,Yes


In [67]:
# space for more columns

## Hotels from Mapusa

In [68]:
df4=df_fn(data_mapusa)

Mapusa


In [69]:
df4

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,OYO 67123 Rangel Goan Guest House,4.0,58,Mapusa,1578,40,947,"Attached Bathroom , Daily Housekeeping , CCTV...",No,No,No,No,No,No,No,Yes,No,No,No,Yes
1,OYO 13914 Graceful studio,4.4,302,Mapusa,5161,40,3097,"Parking Facility , Geyser , AC",Yes,No,Yes,No,No,No,No,No,No,No,No,No
2,OYO Home 47510 Peaceful Stay Parra,3.4,36,Mapusa,6081,40,3649,"Attached Bathroom , CCTV Cameras , Refrigerator",No,No,No,No,No,No,No,Yes,No,No,No,Yes
3,OYO Home 68812 Radiant Studios Parra,3.4,15,Mapusa,4062,40,2437,"Parking Facility , Attached Bathroom , CCTV C...",Yes,No,No,No,No,No,No,Yes,No,No,No,Yes
4,SPOT ON 37412 Hotel Priti International SPOT,3.8,310,Mapusa,1690,40,1014,"Kitchen , Attached Bathroom , CCTV Cameras",No,No,No,No,No,Yes,No,Yes,No,No,No,Yes
5,OYO 15911 Green View Studio,4.2,172,Mapusa,2487,40,1492,"AC , TV , CCTV Cameras",No,No,Yes,No,No,No,Yes,Yes,No,No,No,No
6,OYO 18439 Dazzling 2BHK,4.1,186,Mapusa,3811,40,2287,"Kitchen , Functional Living Area , Attached B...",No,No,No,No,No,Yes,No,No,No,No,No,Yes
7,OYO 19114 Dazzling 1BHK,4.4,55,Mapusa,4156,40,2494,"AC , TV , Functional Living Area",No,No,Yes,No,No,No,Yes,No,No,No,No,No
8,OYO 10496 Exotic 2 BHK,4.6,641,Mapusa,3597,40,2158,"DTH , AC , TV",No,No,Yes,No,No,No,Yes,No,No,No,No,No
9,OYO 18655 Spacious 3BHK,4.5,38,Mapusa,2491,59,1014,"Kitchen , Geyser , AC",No,No,Yes,No,No,Yes,No,No,No,No,No,No


In [70]:
# space for more columns

## Hotels from Dabolim Airport

In [71]:
df5=df_fn(data_dabolimairport)

Dabolim Airport


In [72]:
df5

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,OYO 60494 Dewa,NaN,NaN,Dabolim Airport,2677,40,1606,"Attached Bathroom , Daily Housekeeping , CCTV...",No,No,No,No,No,No,No,Yes,No,No,No,Yes
1,OYO 14532 Hotel Avisha Residency,4.2,254.0,Dabolim Airport,2995,40,1797,"AC , Card Payment , Power backup",No,Yes,Yes,No,No,No,No,No,No,Yes,No,No
2,Collection O 48235 The Business Hotel Vasco St...,4.4,1087.0,Dabolim Airport,3016,40,1810,"Attached Bathroom , CCTV Cameras , Fire-Extin...",No,No,No,No,No,No,No,Yes,No,No,No,Yes
3,OYO 48063 Hotel Royal Palace,4.0,125.0,Dabolim Airport,3053,40,1832,"First Aid , Buzzer/Door Bell , Attached Bathr...",No,No,No,No,No,No,No,No,Yes,No,No,Yes
4,OYO 48159 Bogmalo Heights Holiday,3.5,81.0,Dabolim Airport,3352,40,2011,"Buzzer/Door Bell , Attached Bathroom , Daily ...",No,No,No,No,No,No,No,No,No,No,No,Yes
5,OYO 2844 Dewa Goa Hotel,3.2,155.0,Dabolim Airport,3001,40,1801,"Parking Facility , CCTV Cameras , Geyser",Yes,No,No,No,No,No,No,Yes,No,No,No,No
6,OYO 43615 Hotel Sunshine,3.8,206.0,Dabolim Airport,3057,40,1834,"CCTV Cameras , Fire-Extinguisher , AC",No,No,Yes,No,No,No,No,Yes,No,No,No,No
7,Capital O 2191 Hotel Cliff,3.6,38.0,Dabolim Airport,2571,40,1543,"Parking Facility , CCTV Cameras , Conference ...",Yes,No,No,No,No,No,No,Yes,No,No,No,No
8,OYO 12866 Luxurious Stay,4.4,135.0,Dabolim Airport,2806,40,1684,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
9,OYO 28854 Ocean View Studio Dabolim,3.1,64.0,Dabolim Airport,2331,40,1399,"Parking Facility , Daily Housekeeping , CCTV ...",Yes,No,No,No,No,No,No,Yes,No,No,No,No


In [73]:
# space for more columns

## Hotels from Vasco Da Gama

In [74]:
df6=df_fn(data_vascodagama)

Vasco Da Gama


In [75]:
df6

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,OYO 60494 Dewa,NaN,NaN,Vasco Da Gama,2677,40,1606,"Attached Bathroom , Daily Housekeeping , CCTV...",No,No,No,No,No,No,No,Yes,No,No,No,Yes
1,OYO 14532 Hotel Avisha Residency,4.2,254.0,Vasco Da Gama,2995,40,1797,"AC , Card Payment , Power backup",No,Yes,Yes,No,No,No,No,No,No,Yes,No,No
2,Collection O 48235 The Business Hotel Vasco St...,4.4,1087.0,Vasco Da Gama,3016,40,1810,"Attached Bathroom , CCTV Cameras , Fire-Extin...",No,No,No,No,No,No,No,Yes,No,No,No,Yes
3,OYO 48159 Bogmalo Heights Holiday,3.5,81.0,Vasco Da Gama,3352,40,2011,"Buzzer/Door Bell , Attached Bathroom , Daily ...",No,No,No,No,No,No,No,No,No,No,No,Yes
4,OYO 48063 Hotel Royal Palace,4.0,125.0,Vasco Da Gama,3053,40,1832,"First Aid , Buzzer/Door Bell , Attached Bathr...",No,No,No,No,No,No,No,No,Yes,No,No,Yes
5,OYO 2844 Dewa Goa Hotel,3.2,155.0,Vasco Da Gama,3001,40,1801,"Parking Facility , CCTV Cameras , Geyser",Yes,No,No,No,No,No,No,Yes,No,No,No,No
6,OYO 43615 Hotel Sunshine,3.8,206.0,Vasco Da Gama,3057,40,1834,"CCTV Cameras , Fire-Extinguisher , AC",No,No,Yes,No,No,No,No,Yes,No,No,No,No
7,Capital O 2191 Hotel Cliff,3.6,38.0,Vasco Da Gama,2571,40,1543,"Parking Facility , CCTV Cameras , Conference ...",Yes,No,No,No,No,No,No,Yes,No,No,No,No
8,OYO 12866 Luxurious Stay,4.4,135.0,Vasco Da Gama,2806,40,1684,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
9,OYO 28854 Ocean View Studio Dabolim,3.1,64.0,Vasco Da Gama,2331,40,1399,"Parking Facility , Daily Housekeeping , CCTV ...",Yes,No,No,No,No,No,No,Yes,No,No,No,No


In [76]:
# Space for more columns

## Hotels in Madgaon Railway Station

In [77]:
df7=df_fn(data_madgaon_rs)

Madgaon RS


In [78]:
df7

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,OYO 11950 Relax Holiday Home,4.3,381,Madgaon RS,2900,40,1740,"Parking Facility , CCTV Cameras , Geyser",Yes,No,No,No,No,No,No,Yes,No,No,No,No
1,Capital O 37677 Xec Residency,4.3,1358,Madgaon RS,2026,40,1216,"First Aid , Daily Housekeeping , Elevator",No,No,No,No,No,No,No,No,Yes,No,No,No
2,OYO 6988 Hotel Sincro,3.9,227,Madgaon RS,2250,40,1350,"Parking Facility , Kitchen , First Aid",Yes,No,No,No,No,Yes,No,No,Yes,No,No,No
3,OYO Home 18675 Modern Stay Madgaon Station,3.8,61,Madgaon RS,2787,40,1672,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
4,OYO 41623 Hotel La Flor,3.5,127,Madgaon RS,2206,40,1324,"Kitchen , First Aid , Attached Bathroom",No,No,No,No,No,Yes,No,No,Yes,No,No,Yes
5,Capital O 10765 Hotel Tanish,4.0,291,Madgaon RS,2188,40,1313,"CCTV Cameras , Geyser , Elevator",No,No,No,No,No,No,No,Yes,No,No,No,No
6,OYO 17025 Hotel Go Goa,4.2,148,Madgaon RS,3206,40,1924,"Parking Facility , CCTV Cameras , Conference ...",Yes,No,No,No,No,No,No,Yes,No,No,No,No
7,OYO Home 19549 Modern Stay,4.0,72,Madgaon RS,3841,40,2305,"Parking Facility , Kitchen , Daily Housekeeping",Yes,No,No,No,No,Yes,No,No,No,No,No,No
8,OYO 16160 Spacious Stay,4.2,337,Madgaon RS,2871,40,1723,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
9,OYO 23035 Srinivas Residency,4.1,203,Madgaon RS,1893,46,1014,"First Aid , Daily Housekeeping , Complimentar...",No,No,No,Yes,No,No,No,No,Yes,No,No,No


In [79]:
# spoce for more columns

## Hotels in Morjim

In [80]:
df8=df_fn(data_morjim)

Morjim


In [81]:
df8

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,OYO 70630 Sai Coco Resort Morjim,NaN,NaN,Morjim,1690,40,1014,"Parking Facility , Bar , Attached Bathroom",Yes,No,No,No,No,No,No,No,No,No,No,Yes
1,OYO 8065 Accovah Map5 Inn,3.7,148.0,Morjim,5960,40,3576,"CCTV Cameras , Geyser , Electric Kettle",No,No,No,No,No,No,No,Yes,No,No,No,No
2,Capital O 49261 Belle Velle Resort Deluxe,4.4,334.0,Morjim,3981,40,2389,"Parking Facility , Attached Bathroom , Daily ...",Yes,No,No,No,No,No,No,No,No,No,No,Yes
3,OYO 14644 Riverview Studio,4.4,96.0,Morjim,4557,40,2734,"Parking Facility , AC , TV",Yes,No,Yes,No,No,No,Yes,No,No,No,No,No
4,OYO 22417 Pleasure Inn,3.0,15.0,Morjim,2415,40,1449,"Kitchen , Complimentary Veg Breakfast , Attac...",No,No,No,No,No,Yes,No,No,No,No,No,Yes
5,OYO 60478 Sai Max,4.4,22.0,Morjim,2241,40,1345,"Parking Facility , Wheelchair Accessible , AC",Yes,No,Yes,No,No,No,No,No,No,No,No,No
6,OYO 44616 Morjim Sea Wave,NaN,NaN,Morjim,2485,40,1491,"Power backup , CCTV Cameras",No,No,No,No,No,No,No,Yes,No,Yes,No,No
7,Collection O 15006 Orritel Village,4.0,785.0,Morjim,4465,40,2679,"In-house Restaurant , Card Payment , CCTV Cam...",No,Yes,No,No,No,No,No,Yes,No,No,No,No
8,OYO 10142 Retreat Morjim Resort,3.3,53.0,Morjim,5032,40,3019,"Swimming Pool , Complimentary Breakfast , AC",No,No,Yes,Yes,No,No,No,No,No,No,No,No
9,OYO 65004 La Vihan Resort,3.6,10.0,Morjim,2026,40,1216,"Kitchen , Attached Bathroom , CCTV Cameras",No,No,No,No,No,Yes,No,Yes,No,No,No,Yes


In [82]:
# space for more columns

## Hotels in Panaji

In [124]:
df9=df_fn(data_panjim)

Panaji


In [125]:
df9

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,Collection O 22006 Paradise Inn,4.7,1114,Panaji,3463,45,1891,"CCTV Cameras , Reception , Free Wifi",No,No,No,No,No,No,No,Yes,No,No,No,No
1,Collection O 50349 Hotel Sunheads Nalanda Colony,4.5,579,Panaji,3081,40,1849,"AC , TV , Attached Bathroom",No,No,Yes,No,No,No,Yes,No,No,No,No,Yes
2,Collection O 22247 Casa Paradiso,4.7,1142,Panaji,3070,40,1842,"Power backup , CCTV Cameras",No,No,No,No,No,No,No,Yes,No,Yes,No,No
3,Collection O 18786 Hotel Sona Panaji,4.5,1499,Panaji,3214,44,1803,"AC , TV , CCTV Cameras",No,No,Yes,No,No,No,Yes,Yes,No,No,No,No
4,OYO 11845 Hotel Victoria,4.5,381,Panaji,3078,40,1847,"Elevator , AC , TV",No,No,Yes,No,No,No,Yes,No,No,No,No,No
5,OYO 9518 Hotel Aqua Mira,4.0,145,Panaji,2655,40,1593,"Parking Facility , CCTV Cameras , Geyser",Yes,No,No,No,No,No,No,Yes,No,No,No,No
6,OYO 2863 Hotel 4 Pillar's,4.4,185,Panaji,2857,40,1714,"Complimentary Breakfast , AC , Power backup",No,No,Yes,Yes,No,No,No,No,No,Yes,No,No
7,OYO 35596 Hotel Amar,4.5,89,Panaji,2177,40,1306,"Parking Facility , First Aid , Attached Bathr...",Yes,No,No,No,No,No,No,No,Yes,No,No,Yes
8,Capital O 65108 Viva Goa International,4.1,93,Panaji,5498,40,3299,"Parking Facility , First Aid , Attached Bathr...",Yes,No,No,No,No,No,No,No,Yes,No,No,Yes
9,OYO 14986 Charming 1 BHK,3.7,146,Panaji,5608,40,3365,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No


In [85]:
# space for more columns 

## Hotels in Vasco Da Gama Railway Station

In [86]:
df10=df_fn(data_vascodagama_rs)

Vasco Da Gama RS


In [87]:
df10

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,OYO 60494 Dewa,NaN,NaN,Vasco Da Gama RS,2677,40,1606,"Attached Bathroom , Daily Housekeeping , CCTV...",No,No,No,No,No,No,No,Yes,No,No,No,Yes
1,OYO 14532 Hotel Avisha Residency,4.2,254.0,Vasco Da Gama RS,2995,40,1797,"AC , Card Payment , Power backup",No,Yes,Yes,No,No,No,No,No,No,Yes,No,No
2,Collection O 48235 The Business Hotel Vasco St...,4.4,1087.0,Vasco Da Gama RS,3016,40,1810,"Attached Bathroom , CCTV Cameras , Fire-Extin...",No,No,No,No,No,No,No,Yes,No,No,No,Yes
3,OYO 48159 Bogmalo Heights Holiday,3.5,81.0,Vasco Da Gama RS,3352,40,2011,"Buzzer/Door Bell , Attached Bathroom , Daily ...",No,No,No,No,No,No,No,No,No,No,No,Yes
4,OYO 48063 Hotel Royal Palace,4.0,125.0,Vasco Da Gama RS,3053,40,1832,"First Aid , Buzzer/Door Bell , Attached Bathr...",No,No,No,No,No,No,No,No,Yes,No,No,Yes
5,OYO 2844 Dewa Goa Hotel,3.2,155.0,Vasco Da Gama RS,3001,40,1801,"Parking Facility , CCTV Cameras , Geyser",Yes,No,No,No,No,No,No,Yes,No,No,No,No
6,OYO 43615 Hotel Sunshine,3.8,206.0,Vasco Da Gama RS,3057,40,1834,"CCTV Cameras , Fire-Extinguisher , AC",No,No,Yes,No,No,No,No,Yes,No,No,No,No
7,Capital O 2191 Hotel Cliff,3.6,38.0,Vasco Da Gama RS,2571,40,1543,"Parking Facility , CCTV Cameras , Conference ...",Yes,No,No,No,No,No,No,Yes,No,No,No,No
8,OYO 12866 Luxurious Stay,4.4,135.0,Vasco Da Gama RS,2806,40,1684,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
9,OYO 28854 Ocean View Studio Dabolim,3.1,64.0,Vasco Da Gama RS,2331,40,1399,"Parking Facility , Daily Housekeeping , CCTV ...",Yes,No,No,No,No,No,No,Yes,No,No,No,No


In [88]:
# space for more columns

## Hotels in Ponda

In [89]:
df11=df_fn(data_ponda)

Ponda


In [90]:
df11

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,SPOT ON 38006 Hotel City Central SPOT,3.8,508,Ponda,1276,40,766,"First Aid , Attached Bathroom , Daily Houseke...",No,No,No,No,No,No,No,No,Yes,No,No,Yes
1,OYO 63283 Shakti Palace,3.7,58,Ponda,2026,40,1216,"First Aid , Attached Bathroom , Daily Houseke...",No,No,No,No,No,No,No,No,Yes,No,No,Yes
2,SPOT ON 44584 Hotel Vaishali Residency SPOT,4.1,127,Ponda,836,40,502,"Parking Facility , In-house Restaurant , AC",Yes,No,Yes,No,No,No,No,No,No,No,No,No
3,OYO 9623 Beautiful 5BHK,3.3,19,Ponda,1596,40,958,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
4,OYO 62138 Delightful Stay,4.3,365,Ponda,3352,40,2011,"Parking Facility , Attached Bathroom , Refrig...",Yes,No,No,No,No,No,No,No,No,No,No,Yes
5,OYO 6988 Hotel Sincro,3.9,227,Ponda,2250,40,1350,"Parking Facility , Kitchen , First Aid",Yes,No,No,No,No,Yes,No,No,Yes,No,No,No
6,Capital O 37677 Xec Residency,4.3,1358,Ponda,2026,40,1216,"First Aid , Daily Housekeeping , Elevator",No,No,No,No,No,No,No,No,Yes,No,No,No
7,OYO 49558 Keegan Guest House,4.4,31,Ponda,2050,40,1230,"Bar , Attached Bathroom , Daily Housekeeping",No,No,No,No,No,No,No,No,No,No,No,Yes
8,OYO 29040 Ocean View Studio Dabolim,3.7,101,Ponda,2661,40,1597,"Parking Facility , Daily Housekeeping , CCTV ...",Yes,No,No,No,No,No,No,Yes,No,No,No,No
9,OYO 16160 Spacious Stay,4.2,337,Ponda,2871,40,1723,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No


In [91]:
# space for more columns

## Hotels in Candolim

In [92]:
df12=df_fn(data_candolim)

Candolim


In [93]:
df12

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,Collection O 50217 Cornerstone Candolim,4.1,242.0,Candolim,3231,40,1939,"Attached Bathroom , Daily Housekeeping , CCTV...",No,No,No,No,No,No,No,Yes,No,No,No,Yes
1,OYO Townhouse 065 Calangute Aguada,4.3,759.0,Candolim,5417,40,3250,"CCTV Cameras , Complimentary Breakfast , In-h...",No,No,No,Yes,No,No,No,Yes,No,No,No,No
2,Capital O 61948 Aqua Marina Suite,4.3,220.0,Candolim,9526,40,5716,"Conference Room , Swimming Pool , Wheelchair ...",No,No,No,No,No,No,No,No,No,No,No,No
3,OYO 46308 Flora Villa Candolim,4.4,117.0,Candolim,6251,40,3751,"Attached Bathroom , Daily Housekeeping , CCTV...",No,No,No,No,No,No,No,Yes,No,No,No,Yes
4,OYO Home 66565 Spacious Duplex Calangute,4.3,46.0,Candolim,3055,40,1833,"Parking Facility , Attached Bathroom , CCTV C...",Yes,No,No,No,No,No,No,Yes,No,No,No,Yes
5,Collection O 13853 Shruti Resort Calangute,4.3,495.0,Candolim,4742,40,2845,"Parking Facility , In-house Restaurant , AC",Yes,No,Yes,No,No,No,No,No,No,No,No,No
6,OYO 9364 Modern 2BHK Candolim Beach,3.8,37.0,Candolim,3401,40,2041,"Elevator , Parking Facility , Kitchen",Yes,No,No,No,No,Yes,No,No,No,No,No,No
7,OYO Townhouse 256 Silver Palm Resort,4.6,644.0,Candolim,4320,40,2592,"First Aid , Bar , Attached Bathroom",No,No,No,No,No,No,No,No,Yes,No,No,Yes
8,OYO 10039 Nisu Beach Hotel,3.4,54.0,Candolim,2440,40,1464,"Parking Facility , AC , TV",Yes,No,Yes,No,No,No,Yes,No,No,No,No,No
9,OYO Home 69781 Greenfield View Stay Candolim,4.7,68.0,Candolim,4985,40,2991,"Attached Bathroom , CCTV Cameras , Power backup",No,No,No,No,No,No,No,Yes,No,Yes,No,Yes


In [94]:
# space for more columns

## Hotels in Dona Paula

In [95]:
df13=df_fn(data_donapaula)

Dona Paula


In [96]:
df13

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,OYO 35596 Hotel Amar,4.5,89,Dona Paula,2177,40,1306,"Parking Facility , First Aid , Attached Bathr...",Yes,No,No,No,No,No,No,No,Yes,No,No,Yes
1,Collection O 50349 Hotel Sunheads Nalanda Colony,4.5,579,Dona Paula,3081,40,1849,"AC , TV , Attached Bathroom",No,No,Yes,No,No,No,Yes,No,No,No,No,Yes
2,OYO 12319 Elegant 1BHK Near Vainguinim Beach,4.4,678,Dona Paula,3445,40,2067,"Kitchen , AC , TV",No,No,Yes,No,No,Yes,Yes,No,No,No,No,No
3,OYO 9518 Hotel Aqua Mira,4.0,145,Dona Paula,2655,40,1593,"Parking Facility , CCTV Cameras , Geyser",Yes,No,No,No,No,No,No,Yes,No,No,No,No
4,OYO 61648 Goan Delicacy Deluxe,3.9,76,Dona Paula,2520,40,1512,"Complimentary Breakfast , AC , Refrigerator",No,No,Yes,Yes,No,No,No,No,No,No,No,No
5,Collection O 22006 Paradise Inn,4.7,1114,Dona Paula,3463,45,1891,"CCTV Cameras , Reception , Free Wifi",No,No,No,No,No,No,No,Yes,No,No,No,No
6,OYO 47978 Stay In Caranzalam,3.8,121,Dona Paula,3150,40,1890,"Parking Facility , AC , TV",Yes,No,Yes,No,No,No,Yes,No,No,No,No,No
7,OYO 15007 Casa Amarilla,4.4,294,Dona Paula,5167,40,3100,"Elevator , Parking Facility , Kitchen",Yes,No,No,No,No,Yes,No,No,No,No,No,No
8,OYO 4195 Clausil Inn,4.5,160,Dona Paula,2520,40,1512,"AC , TV , CCTV Cameras",No,No,Yes,No,No,No,Yes,Yes,No,No,No,No
9,OYO 14986 Charming 1 BHK,3.7,146,Dona Paula,5608,40,3365,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No


In [97]:
# space for more columns

## Hotels in Varca

In [98]:
df14=df_fn(data_varca)

Varca


In [99]:
df14

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,Capital O 22969 The Center Court,4.6,391,Varca,3964,46,2158,"First Aid , Daily Housekeeping , Conference R...",No,No,No,No,No,No,No,No,Yes,No,No,No
1,OYO 12036 with Pool 1BHK,4.0,200,Varca,4497,40,2698,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
2,OYO 18662 Beach 2BHK,4.2,84,Varca,3377,40,2026,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
3,OYO 14907 House With Balcony 1BHK,3.8,115,Varca,1690,40,1014,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
4,OYO 65852 Blue Ocean Pearl,3.1,67,Varca,2037,40,1222,"First Aid , Attached Bathroom , Daily Houseke...",No,No,No,No,No,No,No,No,Yes,No,No,Yes
5,OYO 27603 Flamingo's NON,3.4,104,Varca,2062,40,1237,"Attached Bathroom , Daily Housekeeping , CCTV...",No,No,No,No,No,No,No,Yes,No,No,No,Yes
6,OYO 24459 Classy 1BHK Fatrade,4.5,72,Varca,4172,40,2503,"Parking Facility , Kitchen , CCTV Cameras",Yes,No,No,No,No,Yes,No,Yes,No,No,No,No
7,OYO 16582 Oshin Holiday Care,4.3,80,Varca,2940,40,1764,"Parking Facility , Bar , CCTV Cameras",Yes,No,No,No,No,No,No,Yes,No,No,No,No
8,OYO 22853 Elegant Studio Benaulim,4.0,49,Varca,1690,40,1014,"Parking Facility , Bath Tub , Kitchen",Yes,No,No,No,No,Yes,No,No,No,No,No,No
9,OYO Home 13674 Pool View Studio,3.5,72,Varca,1690,40,1014,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No


In [100]:
# space for more columns

## Hotels in Colva

In [101]:
df15=df_fn(data_colva)

Colva


In [102]:
df15

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,OYO 65852 Blue Ocean Pearl,3.1,67,Colva,2037,40,1222,"First Aid , Attached Bathroom , Daily Houseke...",No,No,No,No,No,No,No,No,Yes,No,No,Yes
1,SPOT ON 61966 Effie's Apartment SPOT,3.9,194,Colva,1453,40,872,"Kitchen , First Aid , Attached Bathroom",No,No,No,No,No,Yes,No,No,Yes,No,No,Yes
2,OYO 23544 Elegant 1BHK Betalbetim,4.0,146,Colva,3072,40,1843,"Kitchen , AC , TV",No,No,Yes,No,No,Yes,Yes,No,No,No,No,No
3,OYO 18327 Pool View 1BHK,3.7,64,Colva,1690,40,1014,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
4,OYO 28305 Sunset Holiday Homes,3.2,26,Colva,2892,40,1735,"Daily Housekeeping , AC , Power backup",No,No,Yes,No,No,No,No,No,No,Yes,No,No
5,OYO 10723 Pool View Studio,3.8,158,Colva,4651,40,2791,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
6,OYO 22853 Elegant Studio Benaulim,4.0,49,Colva,1690,40,1014,"Parking Facility , Bath Tub , Kitchen",Yes,No,No,No,No,Yes,No,No,No,No,No,No
7,OYO Home 13674 Pool View Studio,3.5,72,Colva,1690,40,1014,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
8,OYO 10974 Pool View 1BHK,3.8,100,Colva,4651,40,2791,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
9,OYO 19112 Caphina Tourist Cottages,4.3,124,Colva,2380,40,1428,"First Aid , Daily Housekeeping , Complimentar...",No,No,No,Yes,No,No,No,No,Yes,No,No,No


In [103]:
# space for more columns

## Hotels in Vagator

In [104]:
df16=df_fn(data_vagator)

Vagator


In [105]:
df16

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,Capital O 10984 Lagom Comfort,4.6,792.0,Vagator,3925,40,2355,"AC , TV , Card Payment",No,Yes,Yes,No,No,No,Yes,No,No,No,No,No
1,Collection O 30082 Baga Alchemy Creek,4.0,517.0,Vagator,5651,40,3391,"Parking Facility , AC , Card Payment",Yes,Yes,Yes,No,No,No,No,No,No,No,No,No
2,Collection O 15006 Orritel Village,4.0,785.0,Vagator,4465,40,2679,"In-house Restaurant , Card Payment , CCTV Cam...",No,Yes,No,No,No,No,No,Yes,No,No,No,No
3,OYO Home 64158 Sunshine Studios Vagator,4.6,224.0,Vagator,3352,40,2011,"Parking Facility , Attached Bathroom , CCTV C...",Yes,No,No,No,No,No,No,Yes,No,No,No,Yes
4,OYO 60359 Radiant Studio Near Vagator Beach,4.6,145.0,Vagator,3600,40,2160,"Parking Facility , Attached Bathroom , CCTV C...",Yes,No,No,No,No,No,No,Yes,No,No,No,Yes
5,OYO Home 68812 Radiant Studios Parra,3.4,15.0,Vagator,4062,40,2437,"Parking Facility , Attached Bathroom , CCTV C...",Yes,No,No,No,No,No,No,Yes,No,No,No,Yes
6,OYO 18655 Spacious 3BHK,4.5,38.0,Vagator,2491,59,1014,"Kitchen , Geyser , AC",No,No,Yes,No,No,Yes,No,No,No,No,No,No
7,OYO 16095 Luxury Studio Star,4.4,104.0,Vagator,3876,40,2326,"AC , TV , Power backup",No,No,Yes,No,No,No,Yes,No,No,Yes,No,No
8,OYO 70630 Sai Coco Resort Morjim,NaN,NaN,Vagator,1690,40,1014,"Parking Facility , Bar , Attached Bathroom",Yes,No,No,No,No,No,No,No,No,No,No,Yes
9,OYO Home 63293 Shrushthee Studio,4.2,41.0,Vagator,2092,40,1255,"Parking Facility , Attached Bathroom , CCTV C...",Yes,No,No,No,No,No,No,Yes,No,No,No,Yes


In [106]:
# space for more columns

## Hotels in Anjuna

In [107]:
df17=df_fn(data_anjuna)

Anjuna


In [108]:
df17

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,Collection O 30082 Baga Alchemy Creek,4.0,517,Anjuna,5651,40,3391,"Parking Facility , AC , Card Payment",Yes,Yes,Yes,No,No,No,No,No,No,No,No,No
1,Capital O 10984 Lagom Comfort,4.6,792,Anjuna,3925,40,2355,"AC , TV , Card Payment",No,Yes,Yes,No,No,No,Yes,No,No,No,No,No
2,OYO 60359 Radiant Studio Near Vagator Beach,4.6,145,Anjuna,3600,40,2160,"Parking Facility , Attached Bathroom , CCTV C...",Yes,No,No,No,No,No,No,Yes,No,No,No,Yes
3,OYO 7288 Traveller Guest House,4.4,136,Anjuna,3568,40,2141,"AC , TV , CCTV Cameras",No,No,Yes,No,No,No,Yes,Yes,No,No,No,No
4,OYO Home 64158 Sunshine Studios Vagator,4.6,224,Anjuna,3352,40,2011,"Parking Facility , Attached Bathroom , CCTV C...",Yes,No,No,No,No,No,No,Yes,No,No,No,Yes
5,Capital O 64180 Adam's Beach Resort,4.8,252,Anjuna,5026,40,3016,"AC , First Aid , TV",No,No,Yes,No,No,No,Yes,No,Yes,No,No,No
6,Collection O 15006 Orritel Village,4.0,785,Anjuna,4465,40,2679,"In-house Restaurant , Card Payment , CCTV Cam...",No,Yes,No,No,No,No,No,Yes,No,No,No,No
7,OYO 3385 Hotel Baga Bay,4.8,291,Anjuna,3623,40,2174,"In-house Restaurant , Kitchen , AC",No,No,Yes,No,No,Yes,No,No,No,No,No,No
8,OYO Home 64191 Radiant Studios Baga,4.2,117,Anjuna,3162,40,1897,"Parking Facility , Attached Bathroom , CCTV C...",Yes,No,No,No,No,No,No,Yes,No,No,No,Yes
9,OYO Home 66780 Spacious Studios Baga,4.6,157,Anjuna,4535,40,2721,"Attached Bathroom , CCTV Cameras , Power backup",No,No,No,No,No,No,No,Yes,No,Yes,No,Yes


In [109]:
# space for more columns

## Hotels in South Goa

In [110]:
df18=df_fn(data_southgoa)

South Goa


In [111]:
df18

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,OYO 9623 Beautiful 5BHK,3.3,19,South Goa,1596,40,958,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
1,OYO 41615 Grand Villas Saver,3.3,14,South Goa,3593,40,2156,"First Aid , Attached Bathroom , Daily Houseke...",No,No,No,No,No,No,No,No,Yes,No,No,Yes
2,SPOT ON 66016 Grand Villas 2 SPOT,4.3,42,South Goa,2520,40,1512,"Parking Facility , First Aid , Attached Bathr...",Yes,No,No,No,No,No,No,No,Yes,No,No,Yes
3,Capital O 37677 Xec Residency,4.3,1358,South Goa,2026,40,1216,"First Aid , Daily Housekeeping , Elevator",No,No,No,No,No,No,No,No,Yes,No,No,No
4,OYO 11950 Relax Holiday Home,4.3,381,South Goa,2900,40,1740,"Parking Facility , CCTV Cameras , Geyser",Yes,No,No,No,No,No,No,Yes,No,No,No,No
5,OYO Home 18675 Modern Stay Madgaon Station,3.8,61,South Goa,2787,40,1672,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
6,OYO 16160 Spacious Stay,4.2,337,South Goa,2871,40,1723,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No
7,OYO Home 19549 Modern Stay,4.0,72,South Goa,3841,40,2305,"Parking Facility , Kitchen , Daily Housekeeping",Yes,No,No,No,No,Yes,No,No,No,No,No,No
8,OYO 6988 Hotel Sincro,3.9,227,South Goa,2250,40,1350,"Parking Facility , Kitchen , First Aid",Yes,No,No,No,No,Yes,No,No,Yes,No,No,No
9,SPOT ON 38006 Hotel City Central SPOT,3.8,508,South Goa,1276,40,766,"First Aid , Attached Bathroom , Daily Houseke...",No,No,No,No,No,No,No,No,Yes,No,No,Yes


In [112]:
# space for more columns

## Hotels in Arambol

In [113]:
#  df19=df_fn(data_arambol)

In [114]:
# df19

In [115]:
# space for more columns

In [126]:
df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18],ignore_index=True)
df

,Hotel,Rating,Count,Location,Actual Price(₹),Discount(% off),Price(₹),Amenities,Parking,Card Payment,AC,BreakFast,In-house Restaurant,Kitchen,TV,CCTV Cameras,First Aid,Power backup,Bar,Attached Bathroom
0,Collection O 30082 Baga Alchemy Creek,4.0,517.0,Baga Beach,5651,40,3391,"Parking Facility , AC , Card Payment",Yes,Yes,Yes,No,No,No,No,No,No,No,No,No
1,OYO 7288 Traveller Guest House,4.4,136.0,Baga Beach,3568,40,2141,"AC , TV , CCTV Cameras",No,No,Yes,No,No,No,Yes,Yes,No,No,No,No
2,Capital O 64180 Adam's Beach Resort,4.8,252.0,Baga Beach,5026,40,3016,"AC , First Aid , TV",No,No,Yes,No,No,No,Yes,No,Yes,No,No,No
3,OYO Home 65590 Titos Lane 1 Baga Beach,4.0,199.0,Baga Beach,4165,40,2499,"Attached Bathroom , CCTV Cameras , Power backup",No,No,No,No,No,No,No,Yes,No,Yes,No,Yes
4,OYO 3385 Hotel Baga Bay,4.8,291.0,Baga Beach,3623,40,2174,"In-house Restaurant , Kitchen , AC",No,No,Yes,No,No,Yes,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,SPOT ON 40731 Hotel Raviraj Classic SPOT,4.6,902.0,South Goa,2051,40,1231,"Parking Facility , First Aid , Attached Bathr...",Yes,No,No,No,No,No,No,No,Yes,No,No,Yes
356,OYO 41623 Hotel La Flor,3.5,127.0,South Goa,2206,40,1324,"Kitchen , First Aid , Attached Bathroom",No,No,No,No,No,Yes,No,No,Yes,No,No,Yes
357,OYO 42366 Nesh Comfort,3.5,48.0,South Goa,2190,40,1314,"Parking Facility , Complimentary Veg Breakfas...",Yes,No,No,No,No,No,No,No,No,No,No,No
358,OYO 18328 Green View Studio,4.1,133.0,South Goa,2381,40,1429,"Parking Facility , Kitchen , AC",Yes,No,Yes,No,No,Yes,No,No,No,No,No,No


In [127]:
# df.to_csv('df_oyo.csv')

# EOF